In [1]:
import os
import pandas as pd
import re
from dotenv import load_dotenv, find_dotenv
# load .env
load_dotenv(find_dotenv())

True

In [23]:
# Turn off SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # default='warn'

directory = "data"

In [24]:
provinsi_df = pd.read_csv(f"{directory}/raw/provinces.csv", dtype=str)
kabupaten_df = pd.read_csv(f"{directory}/raw/regencies.csv", dtype=str)
# tambahan data dari expert
island_neighbour_df = pd.read_csv(f"{directory}/raw/island_and_neighbours.csv", dtype=str)

In [25]:
provinsi_df.head()

,kode,nama
0,11,ACEH
1,12,SUMATERA UTARA
2,13,SUMATERA BARAT
3,14,RIAU
4,15,JAMBI


In [26]:
kabupaten_df.head()

,kode,nama
0,1101,SIMEULUE
1,1102,ACEH SINGKIL
2,1103,ACEH SELATAN
3,1104,ACEH TENGGARA
4,1105,ACEH TIMUR


In [27]:
island_neighbour_df.head()

,kode,nama,pulau,provinsi_terdekat
0,11,Aceh,Sumatera,Sumatera Utara
1,12,Sumatera Utara,Sumatera,"Aceh, Sumatera Barat, Riau"
2,13,Sumatera Barat,Sumatera,"Sumatera Utara, Riau, Jambi"
3,14,Riau,Sumatera,"Sumatera Barat, Jambi, Kepulauan Riau"
4,15,Jambi,Sumatera,"Riau, Sumatera Selatan, Sumatera Barat"


In [28]:
def get_provinsi(df):
  df['nama'] = df['nama'].str.replace('KEP.', 'KEPULAUAN', regex=False)
  return df

def normalize_kabupaten_nama(kabupaten_nama):
    # Menghilangkan spasi di antara huruf besar yang terpisah oleh satu spasi
    normalized_kabupaten_nama = re.sub(r'\b([A-Z])\s+', r'\1', kabupaten_nama)
    return normalized_kabupaten_nama.strip()

def get_kabupaten(df):
  df['nama'] = df['nama'].apply(normalize_kabupaten_nama)
  return df

def get_relasi_kabupaten_provinsi(df):
  df['provinsi_kode'] = df['kode'].astype(str).str.slice(0, 2)
  df.columns = ['kabupaten_kode', 'kabupaten_nama', 'provinsi_kode']
  selected_df = df[['kabupaten_kode', 'provinsi_kode']]
  return selected_df

def get_relasi_perusahaan_kabupaten_provinsi(df):
  siinas_df = pd.read_csv('../establishments/data/processed/siinas_data.csv')
  siinas_df['kabupaten_nama'] = siinas_df['kabupaten_nama'].str.replace(r'\b(Kabupaten|Kota)\b\s*', '', regex=True)
  siinas_df['kabupaten_nama'] = siinas_df['kabupaten_nama'].str.upper()
  # mengambil hanya id dan kabupaten_nama
  selected_siinas_df = siinas_df[['id', 'kabupaten_nama']]
  selected_df = df[['kode', 'nama']]
  selected_df['provinsi_kode'] = selected_df['kode'].astype(str).str.slice(0, 2)
  selected_df['nama'] = selected_df['nama'].apply(normalize_kabupaten_nama)
  # merge
  merged_df = pd.merge(selected_siinas_df, selected_df, left_on='kabupaten_nama', right_on='nama')
  # ambil hanya id dan kode
  selected_merged_df = merged_df[['id', 'kode', 'provinsi_kode']]
  selected_merged_df.columns = ['id_perusahaan', 'kabupaten_kode', 'provinsi_kode']
  return selected_merged_df

def get_relasi_provinsi_pulau(df):
  selected_df = df[['kode', 'pulau']]
  selected_df.columns = ['provinsi_kode', 'pulau']
  return selected_df

def get_relasi_provinsi_bersebelahan(df):
  mapping_dict = pd.Series(df['kode'].values, index=df['nama']).to_dict()
  # Mengganti nama provinsi bersebelahan dengan kode provinsi yang sesuai
  df['provinsi_tetangga'] = df['provinsi_terdekat'].apply(lambda x: x.split(', '))
  df['provinsi_tetangga'] = df['provinsi_tetangga'].apply(
      lambda x: [mapping_dict.get(neighbor.strip(), neighbor) for neighbor in x]
  )
  # explode menjadi bentuk yang diinginkan
  df_exploded = df[['kode', 'provinsi_tetangga']].explode('provinsi_tetangga')
  # Mengganti nama kolom sesuai format yang diinginkan
  df_exploded.columns = ['provinsi_kode', 'provinsi_tetangga']
  return df_exploded

def save_to_csv(df, filename):
  df.to_csv(f"{directory}/processed/{filename}", index=False)
  print(f"{filename} has been saved to processed")

In [30]:
relasi_provinsi_pulau = island_neighbour_df.copy()
save_to_csv(get_relasi_provinsi_pulau(relasi_provinsi_pulau), 'relasi_provinsi_pulau.csv')
relasi_kabupaten_provinsi = kabupaten_df.copy()
save_to_csv(get_relasi_kabupaten_provinsi(relasi_kabupaten_provinsi), 'relasi_kabupaten_provinsi.csv')
relasi_perusahaan_kabupaten_provinsi = kabupaten_df.copy()
save_to_csv(get_relasi_perusahaan_kabupaten_provinsi(relasi_perusahaan_kabupaten_provinsi), 'relasi_perusahaan_kabupaten_provinsi.csv')
save_to_csv(get_provinsi(provinsi_df), 'provinsi.csv')
save_to_csv(get_kabupaten(kabupaten_df), 'kabupaten.csv')
save_to_csv(get_relasi_provinsi_bersebelahan(island_neighbour_df), 'relasi_provinsi_bersebelahan.csv')

relasi_provinsi_pulau.csv has been saved to processed
relasi_kabupaten_provinsi.csv has been saved to processed
relasi_perusahaan_kabupaten_provinsi.csv has been saved to processed
provinsi.csv has been saved to processed
kabupaten.csv has been saved to processed
relasi_provinsi_bersebelahan.csv has been saved to processed


Karena tidak ada lagi data yang diproses dan sudah sesuai, maka tinggal kita copy-paste ke dalam datasets.

In [31]:
import shutil

def copy_csv(source, destination):
  # Ensure the destination directory exists
  if not os.path.exists(destination):
      os.makedirs(destination)

  # Loop through all files in the source directory
  for filename in os.listdir(source):
      # Check if the file is a CSV file
      if filename.endswith('.csv'):
          # Define full file paths
          full_filename = os.path.join(source, filename)
          if os.path.isfile(full_filename):
              # Copy the CSV file to the destination directory
              shutil.copy(full_filename, destination)

copy_csv(f"{directory}/processed", "../../datasets/regions")